# Geom to Stl

> Converting Geometry Data to STL Print outs.
- toc: true
- prettify: true

<details open>
<summary>

## About: 

</summary>

<div id="viewer"></div> 

### Whats inside?

In this notebook, we build a tool to convert geometry data to an STL, dynamically.

- A geojson map will be loaded in. 
- It's shape will be simplified to be friendly with 3d printing.
- The shape will then be extruded and hollowed out at its base so we can put LED lights inside it.
- The boundary lines will be extruded and inserted into the hollow portion to create walls to prevent light bleaching.
- The boundary lines will be used to carve out lines on the top of the map for clarity

</details>
<details open>
<summary>

## The Function

</summary>

<style> 
  input{
    position: unset;
    width: 200px;
  }
</style>

<form id = 'settingsForm'>
  <div>
    <label>
      <input name="geom" type="text" value="https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"> 
       <!-- <input name="geom" type="text" value="https://raw.githubusercontent.com/benbalter/dc-maps/master/maps/ward-2012.geojson"> -->
      <i>GeoJson URL</i>
    </label>  
  </div> 
  <div>
    <label>
      <input name="simplifyBy" type="number" value="0.0168" min="0" max="1" step=".0001">
      <i>Simplify By</i>
    </label>
  </div> 
  <div id="display_map"></div>
  <!-- 
  <div>
    <label>
      <-- <input name="label" type="text" value="CSA2010"> ->
      <input name="label" type="text" value="WARD"> 
      <i>Label Column</i>
    </label>
  </div> 
  <div>
    <label>
      <input name="convex" type="text" value="DC"> 
      <i>Convex Hull Label</i>
    </label>
  </div> 
  <div>
    <label>
      <!- <input name="feature" type="text" value="name"> ->
      <input name="feature" type="text" value="WARD"> 
      <i>Feature</i>
    </label>
  </div> 
  <div>
    <label>
      <input name="weight" type="number" value=0.025> 
      <i>Weight 0 - .05</i>
    </label>
  </div> 
    -->
  <div>
    <label>
      <input name="depth" type="number" value="5">
      <i>Depth</i>
    </label>
  </div>
  <div>
    <label>
      <input name="wallHeight" type="number" value="100">
      <i>Wall Height</i>
    </label>
  </div>
  <div>
    <label>
      <input name="exteriorWallHeight" type="number" value="140">
      <i>Exterior Wall Height</i>
    </label>
  </div>
  <div>
    <label>
      <input name="exteriorLineWidth" type="number" value="10">
      <i>Exterior Line Width</i>
    </label>
  </div>
  <div>
    <label>
      <input name="interiorLineWidth" type="number" value="5">
      <i>Interior Line Width</i>
    </label>
  </div>
  <div>
    <label>
      <input name="engraveWidth" type="number" value="7">
      <i>Engrave Width</i>
    </label>
  </div>
  <div>
    <label>
      <input name="engraveZpos" type="number" value="7.5">
      <i>Engrave Z Position</i>
    </label>
  </div>
  <div>
    <label>
      <input name="stretchFactor" type="number" value="1.25">
      <i>Stretch Factor</i>
    </label>
  </div>
  <div>
    <label>
      <input name="stepMultiply" type="number" value="5">
      <i>Step Multiply</i>
    </label>
  </div>
  <div>
    <label>
      <input name="precision" type="number" value="1000">
      <i>Precision</i>
    </label>
  </div>
  <div>
    <label>
      <input name="scaleToThisSize" type="number" value="180">
      <i>Scale To This Size</i>
    </label>
  </div>
  <div>
    <label>
      <input name="width" type="number" value="600">
      <i>Width</i>
    </label>
  </div>
  <div>
    <label>
      <input name="height" type="number" value="600">
      <i>Height</i>
    </label>
  </div>
  <div>
    <button type="submit">Submit</button>
  </div>
</form>

<!-- Leaflet -->
<link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"/>

<script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script> 

<!-- Turf -->
<script src='https://unpkg.com/@turf/turf@6/turf.min.js'></script>

<!-- TopoJSON-Client -->
<script src="https://cdn.jsdelivr.net/npm/topojson-client@3"></script>

<!-- TopoJSON-Simplify -->
<script src="https://cdn.jsdelivr.net/npm/topojson-simplify@3"></script>

<!-- TopoJSON-Server -->
<script src="https://cdn.jsdelivr.net/npm/topojson-server@3"></script> 

<script src="https://cdnjs.cloudflare.com/ajax/libs/proj4js/2.6.2/proj4.js"></script> 


<style>
 #display_map { width: 600px; height: 600px; }
</style> 

<script type="module"> 
import { displayMap, initialize } from '/cdn/meshmaps_original.js'; 

window.formValues = {}; 
document.querySelector('form').querySelectorAll('input').forEach(input => {
    const name = input.getAttribute('name');
    const value = input.type === 'number' ? parseFloat(input.value) : input.value;
    formValues[name] = value;
});

function updateWindowValues() {
  const form = document.getElementById('settingsForm');
  form.querySelectorAll('input').forEach(input => {
    const name = input.getAttribute('name');
    const value = input.type === 'number' ? parseFloat(input.value) : input.value;
    window[name] = value;
  });
}
 

function handleFormSubmit(event) {
  event.preventDefault(); // Prevent the default form submission behavior
  updateWindowValues();
  displayMap();
  initialize(); // Call initialize after updating window values
}
document.getElementById('settingsForm').addEventListener('submit', handleFormSubmit);

function handleInputChange(event) {
  updateWindowValues();
  displayMap(); // Call initialize again after updating window values
}
document.querySelector('[name="geom"]').addEventListener('change', handleInputChange);
document.querySelector('[name="simplifyBy"]').addEventListener('change', handleInputChange);

  updateWindowValues();
  displayMap();
  initialize();

</script>

</details>
<details open>
<summary>

## Result

</summary>

<div id='renderer'></div>

</details>